In [33]:
import textwrap
from typing import Dict, List, Union, Optional
from urllib.request import urlretrieve

from bs4 import BeautifulSoup
import pandas as pd
import requests

The US Census Bureau (USCB) runs a massive array of surveys of residents of the United States. After aggregating responses into demographic and/or geographic groups, the USCB publishes those aggregated datasets to its massive public data catalog. I've already developed some tooling to collect and ingest Census datasets into my personal data warehouse and analytics environment, but to map out geographic Census data or join in other geospatial datasets, I also need to collect and ingest spatial files describing the geographic boundaries of Census groupings. Fortunately, the USCB provides shapefiles with these geometries in their TIGER (Topologically Integrated Geographic Encoding and Referencing) data offerings, and in this notebook, I'll synthesize the findings the experiments and research in my last notebook into a data model and data collection tools.

### References:
* [TIGER Technical Docs](https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/tiger-geo-line.2022.html)
* [TIGER Geographic Codes](https://www.census.gov/programs-surveys/geography/technical-documentation/code-lists.html)

Each year, the USCB can add or alter geographic boundaries or connected data features, so they organize TIGER data offering into annual vintages, which can be downloaded either through the USCB web interface or from their [file server interface](https://www2.census.gov/geo/tiger/) (https://www2.census.gov/geo/tiger/).

The functions in the cell below aid in scraping the table off of any of pages linked in that file-tree interface.

In [19]:
#| code-fold: show

def request_page(metadata_url: str) -> requests.models.Response:
    resp = requests.get(metadata_url)
    if resp.status_code == 200:
        return resp
    else:
        raise Exception(f"Couldn't get page metadata for url {metadata_url}")

def scrape_census_ftp_metadata_page(metadata_url: str) -> pd.DataFrame:
    resp = request_page(metadata_url=metadata_url)
    soup = BeautifulSoup(resp.content, "html.parser")
    table = soup.find("table")
    rows = table.find_all("tr")
    table_contents = []
    for row in rows:
        cols = row.find_all("td")
        cols = [col.text.strip() for col in cols]
        table_contents.append(cols)
    table_rows = [el for el in table_contents if len(el) > 0]

    metadata_df = pd.DataFrame(
        [row[1:] for row in table_rows],
        columns=["name", "last_modified", "size", "description"],
    )
    metadata_df["last_modified"] = pd.to_datetime(metadata_df["last_modified"])
    metadata_df["is_dir"] = metadata_df["name"].str.endswith("/")
    metadata_df["clean_name"] = metadata_df["name"].str.replace("/$", "", regex=True)
    metadata_df["is_file"] = (~metadata_df["is_dir"]) & (
        metadata_df["clean_name"] != "Parent Directory"
    )
    while metadata_url.strip().endswith("/"):
        metadata_url = metadata_url[:-1]
    mask = metadata_df["is_file"] | metadata_df["is_dir"]
    metadata_df = metadata_df.loc[mask].copy()
    metadata_df["metadata_url"] = (metadata_url + "/" + metadata_df["clean_name"])
    return metadata_df

`request_page()` makes a GET request for the content at the URL `metadata_url` and returns the response (if successful, i.e. if the response has the "success" HTTP response status code). This function is only called by `scrape_census_ftp_metadata_page()`, but it's separated out to make it easier to test `scrape_census_ftp_metadata_page()`.

In [9]:
#| code-fold: show

def request_page(metadata_url: str) -> requests.models.Response:
    resp = requests.get(metadata_url)
    if resp.status_code == 200:
        return resp
    else:
        raise Exception(f"Couldn't get page metadata for url {metadata_url}")

`scrape_census_ftp_metadata_page()` gets the content from a given Census file-tree page (at URL `metadata_url`), parses it into a convenient and well structured format (a `pandas DataFrame`) with features that aid in filtering to desired rows.

In [21]:
resp = request_page(metadata_url="https://www2.census.gov/geo/tiger/")
soup = BeautifulSoup(resp.content, "html.parser")
table = soup.find("table")
rows = table.find_all("tr")
rows[0:10]

[<tr><th valign="top"><img alt="[ICO]" src="/icons/blank.gif"/></th><th><a href="?C=N;O=D">Name</a></th><th><a href="?C=M;O=A">Last modified</a></th><th><a href="?C=S;O=A">Size</a></th><th><a href="?C=D;O=A">Description</a></th></tr>,
 <tr><th colspan="5"><hr/></th></tr>,
 <tr><td valign="top"><img alt="[PARENTDIR]" src="/icons/back.gif"/></td><td><a href="/geo/">Parent Directory</a></td><td> </td><td align="right">  - </td><td> </td></tr>,
 <tr><td valign="top"><img alt="[   ]" src="/icons/layout.gif"/></td><td><a href="Directory_Contents_ReadMe.pdf">Directory_Contents_ReadMe.pdf</a></td><td align="right">2019-06-25 09:13  </td><td align="right">439K</td><td> </td></tr>,
 <tr><td valign="top"><img alt="[DIR]" src="/icons/folder.gif"/></td><td><a href="GENZ2010/">GENZ2010/</a></td><td align="right">2013-07-24 12:46  </td><td align="right">  - </td><td> </td></tr>,
 <tr><td valign="top"><img alt="[DIR]" src="/icons/folder.gif"/></td><td><a href="GENZ2012/">GENZ2012/</a></td><td align="r

The cell above shows code that extracts HTML `tr` (table-row) elements extracted from the page (as well as what the raw `tr` elements look like before processing), and the cell below shows the end product of `scrape_census_ftp_metadata_page()`.

In [24]:
#| code-fold: show
all_tiger_vintages_df = scrape_census_ftp_metadata_page(
    metadata_url="https://www2.census.gov/geo/tiger/"
)
display(all_tiger_vintages_df.head(5))

,name,last_modified,size,description,is_dir,clean_name,is_file,metadata_url
1,Directory_Contents_ReadMe.pdf,2019-06-25 09:13:00,439K,,False,Directory_Contents_ReadMe.pdf,True,https://www2.census.gov/geo/tiger/Directory_Co...
2,GENZ2010/,2013-07-24 12:46:00,-,,True,GENZ2010,False,https://www2.census.gov/geo/tiger/GENZ2010
3,GENZ2012/,2013-07-24 12:47:00,-,,True,GENZ2012,False,https://www2.census.gov/geo/tiger/GENZ2012
4,GENZ2013/,2014-07-02 08:28:00,-,,True,GENZ2013,False,https://www2.census.gov/geo/tiger/GENZ2013
5,GENZ2014/,2017-07-20 13:31:00,-,,True,GENZ2014,False,https://www2.census.gov/geo/tiger/GENZ2014


In [25]:
#| code-fold: false
print(f"Files onn the top-level TIGER dataset page:               {all_tiger_vintages_df['is_file'].sum():>3}")
print(f"TIGER data offerings on the top-level TIGER dataset page: {all_tiger_vintages_df['is_dir'].sum():>3}")

Files onn the top-level TIGER dataset page:                 1
TIGER data offerings on the top-level TIGER dataset page:  59


The `GENZyyyy` TIGER data offerings are interesting, but the real wealth of geospatial files can be found in the TIGER offerings with names matching the `TIGERyyyy` pattern, and I implemented the `get_tiger_vintages_metadata()` function to get the metadata for these offerings, or **vintages**.

In [32]:
#| code-fold: show

def get_tiger_vintages_metadata() -> pd.DataFrame:
    all_tiger_vintages_df = scrape_census_ftp_metadata_page(
        metadata_url="https://www2.census.gov/geo/tiger/"
    )
    tiger_vintages_df = all_tiger_vintages_df.loc[
        all_tiger_vintages_df["name"].str.contains("^TIGER\d{4}/", regex=True)
    ].copy()
    tiger_vintages_df = tiger_vintages_df.sort_values(by="name", ignore_index=True)
    return tiger_vintages_df

tiger_vintages_df = get_tiger_vintages_metadata()
print(f"Available TIGER vintages:")
for line in textwrap.wrap(", ".join(list(tiger_vintages_df["clean_name"].values)), width=95):
    print(f"    {line}")
display(tiger_vintages_df.head(3))

Available TIGER vintages:
    TIGER1992, TIGER1999, TIGER2002, TIGER2003, TIGER2008, TIGER2009, TIGER2010, TIGER2011,
    TIGER2012, TIGER2013, TIGER2014, TIGER2015, TIGER2016, TIGER2017, TIGER2018, TIGER2019,
    TIGER2020, TIGER2021, TIGER2022


,name,last_modified,size,description,is_dir,clean_name,is_file,metadata_url
0,TIGER1992/,2011-12-19 07:56:00,-,,True,TIGER1992,False,https://www2.census.gov/geo/tiger/TIGER1992
1,TIGER1999/,2012-02-13 11:19:00,-,,True,TIGER1999,False,https://www2.census.gov/geo/tiger/TIGER1999
2,TIGER2002/,2015-05-05 18:37:00,-,,True,TIGER2002,False,https://www2.census.gov/geo/tiger/TIGER2002


Now that we have tools for scraping the any page in the Census's file server, we can use those tools to collect 

In [34]:
class TIGERCatalog:
    def __init__(self):
        self.dataset_vintages = get_tiger_vintages_metadata()

    def get_vintage_metadata(self, year: str) -> pd.DataFrame:
        return self.dataset_vintages.loc[self.dataset_vintages["name"] == f"TIGER{year}/"].copy()

class TIGERVintageCatalog:
    def __init__(self, year: str, catalog: Optional[TIGERCatalog] = None):
        self.year = str(year)
        self.set_catalog(catalog=catalog)

    def set_catalog(self, catalog: Optional[TIGERCatalog]) -> None:
        if catalog is None:
            self.catalog = TIGERCatalog()
        else:
            self.catalog = catalog

    @property
    def vintage_metadata(self):
        return self.catalog.get_vintage_metadata(year=self.year)

    @property
    def vintage_entities(self):
        if len(self.vintage_metadata) == 1:
            tiger_vintage_url = self.vintage_metadata["metadata_url"].values[0]
            return scrape_census_ftp_metadata_page(metadata_url=tiger_vintage_url)
        else:
            raise Exception(f"Failed to get unambiguous metadata (got {self.vintage_metadata})")

    def get_entity_metadata(self, entity_name: str) -> pd.DataFrame:
        return self.vintage_entities.loc[self.vintage_entities["clean_name"] == entity_name].copy()

    def print_entity_names(self):
        entity_names = self.vintage_entities.loc[self.vintage_entities["is_dir"], "clean_name"].values
        print(f"TIGER Entity options for the {self.year} TIGER vintage:")
        for entity_name in entity_names:
            print(f"  - {entity_name}")
        print(f"Entity count: {len(entity_names)}")

class TIGERGeographicEntityVintage:
    def __init__(self, entity_name: str, year: str, catalog: Optional[TIGERCatalog] = None):
        self.entity_name = entity_name
        self.year = str(year)
        self.vintage_catalog = TIGERVintageCatalog(year=year, catalog=catalog)
        self.entity_metadata = self.vintage_catalog.get_entity_metadata(entity_name=self.entity_name)

In [4]:
tiger_catalog = TIGERCatalog()

In [5]:
tiger_catalog.get_vintage_metadata(year="2022")

,name,last_modified,size,description,is_dir,clean_name,is_file,metadata_url
18,TIGER2022/,2022-11-01 08:09:00,-,,True,TIGER2022,False,https://www2.census.gov/geo/tiger/TIGER2022
